In [1]:
import pandas as pd # Do not change these imports
import numpy as np
import math


In [2]:
stock_data = {"Number of Shares" : 250000,
        "Price Per Share" : 60,
        "Debt" : 310000,
        "Excess Cash" : 1000000,
        "FCF" : 500000,
        "High Growth Rate" : 0.1,
        "rwacc": 0.09,
        "re" : 0.11}
stock_data

{'Number of Shares': 250000,
 'Price Per Share': 60,
 'Debt': 310000,
 'Excess Cash': 1000000,
 'FCF': 500000,
 'High Growth Rate': 0.1,
 'rwacc': 0.09,
 're': 0.11}

In [13]:
def growth_rate(data):
    p = data["Price Per Share"]
    fcf = data["FCF"]
    hgr = data["High Growth Rate"]
    fcf4 = fcf * (1+ hgr)**3
    r = data["re"]
    a= fcf/(r - hgr)
    b= ((1+hgr)/(1+r))**3
    pv_hgr = a * (1 - b)
    time_four_pv = (p + pv_hgr) * (1+r)**4
    x = fcf4/time_four_pv
    g = r - x
    return g

In [12]:
print(growth_rate(stock_data))

-0.21733077965999087


In [14]:
def aggregate(path):
    df = pd.read_csv(path) # Do not change this line of code
    
    # Drop Unneccesary columns
    df.drop(['PERMNO', 'COMNAM'], axis=1, inplace = True)
    
    # Replace all NaN values with 0
    df = df.fillna(0)
    
    # Negative is used to represent something else in the data, so abs will remove the negative
    df['PRC'] = abs(df['PRC'])
    
    # Filter out rows where price is zero or DIVAMT is zero
    df = df.loc[df["PRC"] != 0]
    df = df.loc[df["DIVAMT"] != 0]
    
    # Replace date
    df["date"] = df["date"].str.slice(stop = 4)
    
    groups = df.groupby(["CUSIP", "date"], group_keys = False)
    

    price =  groups["PRC"].agg("last")
    divs = list(groups["DIVAMT"].sum())
    cusips = list(price.index.get_level_values(0))
    year = list(price.index.get_level_values(1))
    price = list(price)
    
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "Year" : year,
        "Dividends" : divs,
        "Price" : price
    })
    
    
    # filter singletons
    groups = output.groupby(["CUSIP"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) != 1).reset_index().drop(['index'], axis = 1)
    
    return filtered

In [18]:
def overvalued(annual_data):
    #Sorting Dataframe
    groups = dataframe.groupby("CUSIP")
    #iterate through dataframe by CUSIP
        #internally go through and get avg
    over_count = 0
    total = 0
    for index, frame in groups:
        count = 0
        sec_avg = 0
        rows = 1
        final_price = 0
        first_year = frame.loc(1, "Year")
        for index, row in frame.iterrows():
            if first_year == frame.loc(rows, "Year"):
                rows = rows + 1
            else:
                x = frame.loc[rows, "Dividends"]
                y = frame.loc[rows-1, "Dividends"]
                sec_avg = sec_avg + x/y
                count = count + 1
                final_price = frame.loc[rows, "Price"]
        div_growth = sec_avg/count
        ideal_price = frame.loc(1, "Dividends") * (1+div_growth)**count + final_price
        if frame.loc(1, "Price") > ideal_price:
            total = total + 1
        else:
            total = total + 1
            over_count= over_count + 1
        z = over_count / total    
    return z